In [ ]:
import os
import re
import gzip
import shutil
import pandas as pd

In [ ]:
## Test if index column needs to be removed

df1 = pd.read_csv(
    "/gpfs/project/anste145/input_files/buckeye_data/updated_eventfiles/s01.tsv",
    low_memory=True,
    dtype=({"outcomes": "category"}),
    engine="c",
    sep="\t",
)
df1
df1.drop(columns="Unnamed: 0")
# --> it does

In [ ]:
## Fix event files by making them actuall tsv and removing the index column.

path = "/gpfs/project/anste145/input_files/buckeye_data/updated_eventfiles/"
all_ = []

for file in os.listdir(path):
    df = pd.read_csv(
        path + file,
        low_memory=True,
        dtype=({"outcomes": "category"}),
        engine="c",
        sep="\t",
    )
    df = df.drop(columns="Unnamed: 0")
    df.to_csv(
        "/gpfs/project/anste145/input_files/buckeye_data/updated_eventfiles/" + file,
        sep="\t",
        index=False,
    )

In [ ]:
## Merging the individual files again.

path = "/gpfs/project/anste145/input_files/buckeye_data/updated_eventfiles/"
all_ = []

for file in os.listdir(path):
    df = pd.read_csv(
        path + file,
        low_memory=True,
        dtype=({"outcomes": "category"}),
        engine="c",
        sep="\t",
    )
    all_.append(df)

big_df = pd.concat(all_)
concat_w = big_df["outcomes"].tolist()

In [ ]:
## Testing if any words are missing.

regression_words = pd.read_csv(
    "/gpfs/project/anste145/input_files/buckeye_data/regression_data.csv",
    usecols=["wordID"],
    dtype=({"wordID": "category"}),
    engine="c",
    low_memory=True,
)
regr_w = regression_words["wordID"].tolist()
missing = [w for w in regr_w if w not in concat_w]
len(missing)
# --> No words are missing.

In [ ]:
## Writing final event file to file and creating a gzipped version for running the NDL model.

big_df.to_csv(
    "/gpfs/project/anste145/input_files/buckeye_data/final_eventfile_buckeye.tsv",
    sep="\t",
    index=False,
)

with open(
    "/gpfs/project/anste145/input_files/buckeye_data/final_eventfile_buckeye.tsv", "rb"
) as f:
    with gzip.open(
        "/gpfs/project/anste145/input_files/buckeye_data/final_eventfile_buckeye.gz",
        "wb",
    ) as output:
        shutil.copyfileobj(f, output)